In [4]:
import matplotlib.image as mp_image
import matplotlib.pyplot as plt
import subprocess
import numpy as np
import pandas as pd
import tensorflow as tf
from functools import partial
sess = tf.Session()

In [5]:
def extract_filename(full_name, filename_extension):
    return full_name.split(sep="."+filename_extension)[0].split(sep='/')[-1]

extract_png_filename = partial(extract_filename, filename_extension='png')

## 讀取訓練資料到陣列

In [7]:
# ground truth
df_UNBC_pain_score = pd.read_csv('./UNBC_pain_score_table.csv') # 讀取疼痛分數資料
df_UNBC_pain_score.sort_values(by=['PSPI'], ascending=False, inplace=True) # 高分排前面
df_UNBC_pain_score.reset_index(drop=True, inplace=True) # 更新 index

In [9]:
# feed training data

# 建構檔案路徑
# croped_face_folder = '../datalake/crop_face_from_UNBC_pain/'
file_ext = '.png'
# 讀取目錄底下所有檔案絕對路徑
paths_croped_face_img = subprocess.getoutput('find ~/下載/crop_face_from_UNBC_pain/ -type f -name "*.png"').split('\n')
print("count = {}".format(+len(paths_croped_face_img)))
# paths_croped_face_img

count = 46410


In [10]:
# 只使用有被偵測到臉的
df_croped_face_img = pd.DataFrame(
                        data = {'full_name':paths_croped_face_img, 
                                'file_name':[i for i in map(extract_png_filename, paths_croped_face_img)]})

In [11]:
# 合併檔案路徑表和分數表，之後只用這張表，清空不必要變數
df_croped_face_img = df_croped_face_img.merge(right=df_UNBC_pain_score, on='file_name', how='inner')
paths_croped_face_img = []
df_UNBC_pain_score = []
print(df_croped_face_img.index.size)

46410


In [12]:
# tensorflow 讀取影像
training_dataset = []
read_fail_set = []
for img_full_name in df_croped_face_img['full_name']:
    try:
        raw_img = tf.image.decode_png(tf.read_file(img_full_name), channels=1).eval(session=sess)
    except:
        read_fail_set.append(img_full_name)
        print(img_full_name, len(read_fail_set))

    resize_img = tf.image.resize_images(raw_img, (32,32))
    reshape_img = tf.reshape(resize_img, [-1])
    training_dataset.append(reshape_img.eval(session=sess))
    if(0 == len(training_dataset)%1000):
        print("read img {} ...".format(len(training_dataset)))

read img 1000 ...


KeyboardInterrupt: 

In [17]:
len(training_dataset[0])
# training_dataset

1024

## Neural Network construct

In [10]:
# 定義輸出入變數
input_layer = tf.placeholder(dtype=tf.float32, shape=[None, 1024], name='input_layer')
ground_truth = tf.placeholder(dtype=tf.float32, shape=[None, 1], name='ground_truth')

In [11]:
# 建構神經網路

# 單層建構函式
def make_layer(inputs, in_size, out_size, activation_function=None):
    Weights = tf.Variable(tf.random_normal([in_size, out_size]))
    biases = tf.Variable(tf.zeros([1, out_size]) + 0.1)
    Wx_plus_b = tf.add(tf.matmul(inputs, Weights), biases)
    if activation_function is None:
        outputs = Wx_plus_b
    else:
        outputs = activation_function(Wx_plus_b)
    return outputs

in_size = 1024
out_size = 1 

# 定義輸出入變數
xs = tf.placeholder(tf.float32, [None, in_size])
ys = tf.placeholder(tf.float32, [None, out_size])

# 定義多層流程
my_activation_function = tf.nn.relu # 統一中間層活化函數
layer_hidden_1 = make_layer(xs, in_size, 32, activation_function=my_activation_function)
layer_output   = make_layer(layer_hidden_1, 32, out_size) 

In [12]:
# 定義損失函數
loss = tf.reduce_mean(tf.reduce_sum(tf.square(ys - layer_output)))

In [32]:
# 定義學習方法
train_step = tf.train.GradientDescentOptimizer(0.0001).minimize(loss)

In [33]:
# 變數初始化
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [ ]:
# 訓練
for i in range(number_sub_sample*20):
    _, cost = sess.run([train_step, loss], 
                           feed_dict = {
                                        xs: training_dataset,
                                        ys: [ [df_croped_face_img.iloc[i]['PSPI']]
                                                    for i in itretor_pain_img ]
                                       })
    # 發散的話就不要算了
    if(np.isnan(cost)):
        print(cost)
        break
        
    if(i%10==0):
        print(i, cost)